In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from numpy import *
from sympy import *
import plotly.offline as py
import plotly.graph_objs as go
%matplotlib inline
init_printing()

In [2]:
Variables=symbols("x1 y1 z1 x2 y2 z2")
x1,y1,z1,x2,y2,z2=Variables
Time=symbols("t")
t=Time
Parameters=symbols("sigma beta rho a b c eps")
sigma,beta,rho,a,b,c,eps=Parameters

### Lorenz system

In [3]:
F=Matrix([sigma*(y1-x1),x1*(rho-z1)-y1,x1*y1-beta*z1,
          sigma*(y2-x2),x2*(rho-z2)-y2,x2*y2-beta*z2])
F

⎡  σ⋅(-x₁ + y₁)  ⎤
⎢                ⎥
⎢x₁⋅(ρ - z₁) - y₁⎥
⎢                ⎥
⎢ -β⋅z₁ + x₁⋅y₁  ⎥
⎢                ⎥
⎢  σ⋅(-x₂ + y₂)  ⎥
⎢                ⎥
⎢x₂⋅(ρ - z₂) - y₂⎥
⎢                ⎥
⎣ -β⋅z₂ + x₂⋅y₂  ⎦

In [4]:
zeta=Matrix([0,0,0,
             0,0,0])
#zeta=eps*zeta
zeta

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

### Synchrony

In [5]:
phi=Matrix([(x2-a*t*x1),(y2-b*t*y1),(z2-c*t*z1)])
phi

⎡-a⋅t⋅x₁ + x₂⎤
⎢            ⎥
⎢-b⋅t⋅y₁ + y₂⎥
⎢            ⎥
⎣-c⋅t⋅z₁ + z₂⎦

In [6]:
N=simulation.VectorDivergence(phi,Variables)
N

⎡-a⋅t   0     0    1  0  0⎤
⎢                         ⎥
⎢ 0    -b⋅t   0    0  1  0⎥
⎢                         ⎥
⎣ 0     0    -c⋅t  0  0  1⎦

In [7]:
on_manifold = [(x2,a*t*x1),(y2,b*t*y1),(z2,c*t*z1)]

## Invariance

In [8]:
# NF=(N*(F+zeta)).subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
# simplify(NF)
NF=N*(F+zeta)
NF
#simplify(NF)

⎡     -a⋅σ⋅t⋅(-x₁ + y₁) + σ⋅(-x₂ + y₂)     ⎤
⎢                                          ⎥
⎢-b⋅t⋅(x₁⋅(ρ - z₁) - y₁) + x₂⋅(ρ - z₂) - y₂⎥
⎢                                          ⎥
⎣   -β⋅z₂ - c⋅t⋅(-β⋅z₁ + x₁⋅y₁) + x₂⋅y₂    ⎦

In [9]:
dPhi=diff(phi,Time)
dPhi

⎡-a⋅x₁⎤
⎢     ⎥
⎢-b⋅y₁⎥
⎢     ⎥
⎣-c⋅z₁⎦

In [10]:
eqn=(NF+dPhi)
simplify(eqn)

⎡      a⋅σ⋅t⋅(x₁ - y₁) - a⋅x₁ - σ⋅(x₂ - y₂)       ⎤
⎢                                                 ⎥
⎢-b⋅t⋅(x₁⋅(ρ - z₁) - y₁) - b⋅y₁ + x₂⋅(ρ - z₂) - y₂⎥
⎢                                                 ⎥
⎣    -β⋅z₂ + c⋅t⋅(β⋅z₁ - x₁⋅y₁) - c⋅z₁ + x₂⋅y₂    ⎦

In [11]:
zeta[3]=-(eqn)[0]
zeta[4]=-(eqn)[1]
zeta[5]=-(eqn)[2]
simplify(zeta)

⎡                       0                        ⎤
⎢                                                ⎥
⎢                       0                        ⎥
⎢                                                ⎥
⎢                       0                        ⎥
⎢                                                ⎥
⎢     -a⋅σ⋅t⋅(x₁ - y₁) + a⋅x₁ + σ⋅(x₂ - y₂)      ⎥
⎢                                                ⎥
⎢b⋅t⋅(x₁⋅(ρ - z₁) - y₁) + b⋅y₁ - x₂⋅(ρ - z₂) + y₂⎥
⎢                                                ⎥
⎣    β⋅z₂ - c⋅t⋅(β⋅z₁ - x₁⋅y₁) + c⋅z₁ - x₂⋅y₂    ⎦

In [12]:
#zeta = zeta.subs(on_manifold)

In [13]:
NF=N*(F+zeta)#.subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
simplify(NF)
simplify((NF+dPhi).subs(on_manifold))

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

### Stability

In [14]:
chi=Matrix([0,0,0,
            -(x2-a*t*x1),-(y2-b*t*y1),-(z2-c*t*z1)])
chi

⎡     0     ⎤
⎢           ⎥
⎢     0     ⎥
⎢           ⎥
⎢     0     ⎥
⎢           ⎥
⎢a⋅t⋅x₁ - x₂⎥
⎢           ⎥
⎢b⋅t⋅y₁ - y₂⎥
⎢           ⎥
⎣c⋅t⋅z₁ - z₂⎦

In [15]:
del_zeta=simulation.VectorDivergence(zeta,Variables)
del_zeta

⎡     0           0          0          0      0   0 ⎤
⎢                                                    ⎥
⎢     0           0          0          0      0   0 ⎥
⎢                                                    ⎥
⎢     0           0          0          0      0   0 ⎥
⎢                                                    ⎥
⎢ -a⋅σ⋅t + a    a⋅σ⋅t        0          σ     -σ   0 ⎥
⎢                                                    ⎥
⎢b⋅t⋅(ρ - z₁)  -b⋅t + b   -b⋅t⋅x₁    -ρ + z₂   1   x₂⎥
⎢                                                    ⎥
⎣   c⋅t⋅y₁      c⋅t⋅x₁   -β⋅c⋅t + c    -y₂    -x₂  β ⎦

In [16]:
del_chi=simulation.VectorDivergence(chi,Variables)
del_chi

⎡ 0    0    0   0   0   0 ⎤
⎢                         ⎥
⎢ 0    0    0   0   0   0 ⎥
⎢                         ⎥
⎢ 0    0    0   0   0   0 ⎥
⎢                         ⎥
⎢a⋅t   0    0   -1  0   0 ⎥
⎢                         ⎥
⎢ 0   b⋅t   0   0   -1  0 ⎥
⎢                         ⎥
⎣ 0    0   c⋅t  0   0   -1⎦

In [17]:
del_F=simulation.VectorDivergence(F,Variables)
del_F

⎡  -σ    σ    0     0     0    0 ⎤
⎢                                ⎥
⎢ρ - z₁  -1  -x₁    0     0    0 ⎥
⎢                                ⎥
⎢  y₁    x₁  -β     0     0    0 ⎥
⎢                                ⎥
⎢  0     0    0     -σ    σ    0 ⎥
⎢                                ⎥
⎢  0     0    0   ρ - z₂  -1  -x₂⎥
⎢                                ⎥
⎣  0     0    0     y₂    x₂  -β ⎦

In [18]:
(del_F+del_zeta+del_chi)

⎡       -σ           σ            0          0   0   0 ⎤
⎢                                                      ⎥
⎢     ρ - z₁         -1          -x₁         0   0   0 ⎥
⎢                                                      ⎥
⎢       y₁           x₁           -β         0   0   0 ⎥
⎢                                                      ⎥
⎢-a⋅σ⋅t + a⋅t + a  a⋅σ⋅t          0          -1  0   0 ⎥
⎢                                                      ⎥
⎢  b⋅t⋅(ρ - z₁)      b         -b⋅t⋅x₁       0   -1  0 ⎥
⎢                                                      ⎥
⎣     c⋅t⋅y₁       c⋅t⋅x₁  -β⋅c⋅t + c⋅t + c  0   0   -1⎦

In [19]:
diff(N,t)*N.transpose()

⎡ 2              ⎤
⎢a ⋅t   0     0  ⎥
⎢                ⎥
⎢       2        ⎥
⎢ 0    b ⋅t   0  ⎥
⎢                ⎥
⎢             2  ⎥
⎣ 0     0    c ⋅t⎦

In [20]:
Jac = N*(del_F+del_zeta+del_chi)*N.transpose() + diff(N,t)*N.transpose()
Jac = simplify(Jac)
Jac.subs(on_manifold)

⎡   2  2                              ⎤
⎢- a ⋅t  - 1       0            0     ⎥
⎢                                     ⎥
⎢                2  2                 ⎥
⎢     0       - b ⋅t  - 1       0     ⎥
⎢                                     ⎥
⎢                             2  2    ⎥
⎣     0            0       - c ⋅t  - 1⎦

In [22]:
Jac.eigenvals()

### coupling

In [23]:
chi

⎡     0     ⎤
⎢           ⎥
⎢     0     ⎥
⎢           ⎥
⎢     0     ⎥
⎢           ⎥
⎢a⋅t⋅x₁ - x₂⎥
⎢           ⎥
⎢b⋅t⋅y₁ - y₂⎥
⎢           ⎥
⎣c⋅t⋅z₁ - z₂⎦

In [24]:
zeta

⎡                       0                        ⎤
⎢                                                ⎥
⎢                       0                        ⎥
⎢                                                ⎥
⎢                       0                        ⎥
⎢                                                ⎥
⎢     a⋅σ⋅t⋅(-x₁ + y₁) + a⋅x₁ - σ⋅(-x₂ + y₂)     ⎥
⎢                                                ⎥
⎢b⋅t⋅(x₁⋅(ρ - z₁) - y₁) + b⋅y₁ - x₂⋅(ρ - z₂) + y₂⎥
⎢                                                ⎥
⎣   β⋅z₂ + c⋅t⋅(-β⋅z₁ + x₁⋅y₁) + c⋅z₁ - x₂⋅y₂    ⎦

In [25]:
simplify((chi+zeta))#.subs([(x2,a*t*x1),(y2,b*t*y1),(z2,c*t*z1)])))

⎡                          0                           ⎤
⎢                                                      ⎥
⎢                          0                           ⎥
⎢                                                      ⎥
⎢                          0                           ⎥
⎢                                                      ⎥
⎢ -a⋅σ⋅t⋅(x₁ - y₁) + a⋅t⋅x₁ + a⋅x₁ + σ⋅(x₂ - y₂) - x₂  ⎥
⎢                                                      ⎥
⎢ b⋅t⋅y₁ + b⋅t⋅(x₁⋅(ρ - z₁) - y₁) + b⋅y₁ - x₂⋅(ρ - z₂) ⎥
⎢                                                      ⎥
⎣β⋅z₂ + c⋅t⋅z₁ - c⋅t⋅(β⋅z₁ - x₁⋅y₁) + c⋅z₁ - x₂⋅y₂ - z₂⎦

### conclusion

* Lorenz system dont go along nwith temporal synchronisation
    - either annihilate original system
    - or deal with incomputable eigenvalue